In [1]:
from agential.cog.agent.react import ReActAgent
from agential.cog.prompts.agent.react import REACT_INSTRUCTION_GSM8K
from agential.cog.prompts.benchmark.gsm8k import GSM8K_FEWSHOT_EXAMPLES_REACT

from langchain_openai import ChatOpenAI

import warnings
warnings.filterwarnings('ignore')

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()

In [2]:
agent = ReActAgent(llm, mode={"math": "gsm8k"})

In [3]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
warnings.filterwarnings('ignore')

out = agent.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_GSM8K,
)

<===================================================================>
Answer a math question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be two types:
(1) Calculate[<insert your code here>], which implements code to answer the math question.
(2) Finish[<insert your answer here>], which returns the code to answer the math question and finishes the task.
You have a maximum of 6 steps.

Here are some examples:
Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
Thought 1: First, I need to find out how many lollipops Jason gave to Denny.
Action 1: Calculate[
```python
jason_lollipops_initial = 20
jason_lollipops_after = 12
answer = jason_lollipops_initial - jason_lollipops_after
```
]
Observation 1: 
```python
jason_lollipops_initial = 20
jason_lollipops_after = 12
answer = jason_lollipops_initial - jason_lo